In [1]:
import torch

# Check for available devices
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use CUDA if available (GPU acceleration)
elif torch.backends.mps.is_available():
    device = torch.device("mps")   # Use MPS if available (macOS with Metal)
else:
    device = torch.device("cpu")   # Default to CPU if neither CUDA nor MPS are available

print(f"Using device: {device}")


Using device: cuda


In [2]:
# 1 folder in train and another 1 in val
from tqdm import tqdm
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copy2

# Paths
dataset_path = "../Dataset/GTSRB"
train_csv_path = os.path.join(dataset_path, "Train.csv")
train_images_path = os.path.join(dataset_path, "Train")

# YOLO directories for train and val
yolo_train_path = os.path.join(dataset_path, "YOLOv8", "train")
yolo_val_path = os.path.join(dataset_path, "YOLOv8", "val")
os.makedirs(yolo_train_path, exist_ok=True)
os.makedirs(yolo_val_path, exist_ok=True)

# Create a folder 'images' inside train and val
train_images_folder = os.path.join(yolo_train_path, "images")
val_images_folder = os.path.join(yolo_val_path, "images")
os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(val_images_folder, exist_ok=True)

# Load the train CSV
train_df = pd.read_csv(train_csv_path)

# Split dataset (80% train, 20% val) while stratifying by class
train_df, val_df = train_test_split(
    train_df, test_size=0.2, stratify=train_df["ClassId"], random_state=42
)

# Save new CSVs for tracking
train_csv_output = os.path.join(dataset_path, "train_split.csv")
val_csv_output = os.path.join(dataset_path, "val_split.csv")
train_df.to_csv(train_csv_output, index=False)
val_df.to_csv(val_csv_output, index=False)

# Function to copy files to the 'images' directory in train or val
def move_files(df, output_dir):
    for _, row in df.iterrows():
        # Construct source and destination paths
        src = os.path.join(dataset_path, row["Path"])
        
        # Set destination folder to a single 'images' folder
        dst = os.path.join(output_dir, "images", os.path.basename(row["Path"]))

        # Create destination directory if it doesn't exist
        os.makedirs(os.path.dirname(dst), exist_ok=True)

        # Debugging: Check paths
        print(f"Source: {src}")
        print(f"Destination: {dst}")
        print("Success!")

        # Copy the file
        if os.path.exists(src):  # Ensure the source file exists
            copy2(src, dst)
        else:
            print(f"File not found: {src}")  # Log missing files

# Move train and validation images into a single 'images' folder
move_files(train_df, yolo_train_path)
move_files(val_df, yolo_val_path)


Source: ../Dataset/GTSRB\Train/12/00012_00031_00018.png
Destination: ../Dataset/GTSRB\YOLOv8\train\images\00012_00031_00018.png
Success!
Source: ../Dataset/GTSRB\Train/5/00005_00031_00001.png
Destination: ../Dataset/GTSRB\YOLOv8\train\images\00005_00031_00001.png
Success!
Source: ../Dataset/GTSRB\Train/12/00012_00069_00028.png
Destination: ../Dataset/GTSRB\YOLOv8\train\images\00012_00069_00028.png
Success!
Source: ../Dataset/GTSRB\Train/13/00013_00052_00022.png
Destination: ../Dataset/GTSRB\YOLOv8\train\images\00013_00052_00022.png
Success!
Source: ../Dataset/GTSRB\Train/18/00018_00017_00024.png
Destination: ../Dataset/GTSRB\YOLOv8\train\images\00018_00017_00024.png
Success!
Source: ../Dataset/GTSRB\Train/38/00038_00059_00025.png
Destination: ../Dataset/GTSRB\YOLOv8\train\images\00038_00059_00025.png
Success!
Source: ../Dataset/GTSRB\Train/12/00012_00034_00026.png
Destination: ../Dataset/GTSRB\YOLOv8\train\images\00012_00034_00026.png
Success!
Source: ../Dataset/GTSRB\Train/4/00004_000

In [3]:
# import os
# import pandas as pd
# from tqdm import tqdm

# # Paths
# dataset_path = "/Users/rattanakpanhakong/Desktop/Traffic-Sign-Detection/Dataset/GTSRB"
# train_csv_path = os.path.join(dataset_path, "train.csv")
# test_csv_path = os.path.join(dataset_path, "test.csv")
# train_images_path = os.path.join(dataset_path, "Train")
# yolo_train_path = os.path.join(dataset_path, "YOLOv8", "train")
# yolo_val_path = os.path.join(dataset_path, "YOLOv8", "val")
# labels_folder_path = os.path.join(yolo_train_path, "labels")  # Directory for the labels

# # Create directories for YOLO format
# os.makedirs(yolo_train_path, exist_ok=True)
# os.makedirs(yolo_val_path, exist_ok=True)
# os.makedirs(labels_folder_path, exist_ok=True)  # Create labels folder

# # Load CSV
# train_df = pd.read_csv(train_csv_path)

# def convert_to_yolo_format(row):
#     """
#     Convert bounding box data to YOLO format.
#     """
#     img_width = row["Width"]
#     img_height = row["Height"]
#     x1 = row["Roi.X1"]
#     y1 = row["Roi.Y1"]
#     x2 = row["Roi.X2"]
#     y2 = row["Roi.Y2"]

#     # YOLO format
#     x_center = ((x1 + x2) / 2) / img_width
#     y_center = ((y1 + y2) / 2) / img_height
#     width = (x2 - x1) / img_width
#     height = (y2 - y1) / img_height

#     return row["ClassId"], x_center, y_center, width, height

# # Convert train data
# for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
#     class_id, x_center, y_center, width, height = convert_to_yolo_format(row)
#     image_path = os.path.join(train_images_path, row["Path"])
#     txt_file = os.path.join(labels_folder_path, f"{os.path.splitext(os.path.basename(row['Path']))[0]}.txt")

#     # Copy image to YOLO train directory
#     os.makedirs(os.path.dirname(txt_file), exist_ok=True)
#     with open(txt_file, "w") as f:
#         f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")




import os
import pandas as pd
from tqdm import tqdm

# Paths
dataset_path = "../Dataset/GTSRB"
split_train_csv_path = os.path.join(dataset_path, "train_split.csv")
split_val_csv_path = os.path.join(dataset_path, "val_split.csv")
train_images_path = os.path.join(dataset_path, "Train")
yolo_train_path = os.path.join(dataset_path, "YOLOv8", "train")
yolo_val_path = os.path.join(dataset_path, "YOLOv8", "val")

# Directories for images and labels
train_images_folder = os.path.join(yolo_train_path, "images")
val_images_folder = os.path.join(yolo_val_path, "images")
train_labels_folder = os.path.join(yolo_train_path, "labels")
val_labels_folder = os.path.join(yolo_val_path, "labels")

# Create directories for YOLO format (train and val)
os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(val_images_folder, exist_ok=True)
os.makedirs(train_labels_folder, exist_ok=True)
os.makedirs(val_labels_folder, exist_ok=True)

# Load CSV
train_df = pd.read_csv(split_train_csv_path)
val_df = pd.read_csv(split_val_csv_path)

def convert_to_yolo_format(row):
    """
    Convert bounding box data to YOLO format.
    """
    img_width = row["Width"]
    img_height = row["Height"]
    x1 = row["Roi.X1"]
    y1 = row["Roi.Y1"]
    x2 = row["Roi.X2"]
    y2 = row["Roi.Y2"]

    # YOLO format
    x_center = ((x1 + x2) / 2) / img_width
    y_center = ((y1 + y2) / 2) / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height

    return row["ClassId"], x_center, y_center, width, height

def save_yolo_labels(df, image_folder, labels_folder):
    """
    Save YOLO labels for each image based on the dataframe.
    """
    for _, row in tqdm(df.iterrows(), total=len(df)):
        class_id, x_center, y_center, width, height = convert_to_yolo_format(row)
        image_path = os.path.join(dataset_path, row["Path"])
        base_name = os.path.splitext(os.path.basename(row["Path"]))[0]
        
        # Choose the correct label file path based on train or val
        label_file = os.path.join(labels_folder, f"{base_name}.txt")

        # Copy image to appropriate YOLO image directory
        dst_image = os.path.join(image_folder, os.path.basename(row["Path"]))
        os.makedirs(os.path.dirname(dst_image), exist_ok=True)
        with open(label_file, "w") as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

        # If image exists, copy it to the correct folder
        if os.path.exists(image_path):
            copy2(image_path, dst_image)
        else:
            print(f"Image not found: {image_path}")

# Save train and validation labels and images
save_yolo_labels(train_df, train_images_folder, train_labels_folder)
save_yolo_labels(val_df, val_images_folder, val_labels_folder)


100%|██████████| 7842/7842 [00:03<00:00, 2013.68it/s]


In [5]:
from ultralytics import YOLO

# Load model
model = YOLO("yolov8n.pt")

model = model.to(device)
print(device)

# Train model
model.train(data="dataset.yaml", epochs=10, batch=16, imgsz=640)

model.save()

cuda
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dyn

100%|██████████| 755k/755k [00:00<00:00, 4.59MB/s]


Overriding model.yaml nc=80 with nc=43

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

100%|██████████| 5.35M/5.35M [00:01<00:00, 5.59MB/s]


AMP: checks passed 


...

      10/10      2.27G     0.4018      0.227      1.106          7        640: 100%|██████████| 1961/1961 [03:43<00:00,  8.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 246/246 [00:31<00:00,  7.84it/s]

                   all       7842       7842      0.993      0.993      0.994      0.946



10 epochs completed in 0.737 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.52  Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 168 layers, 3,014,033 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 246/246 [00:31<00:00,  7.90it/s]


                   all       7842       7842      0.993      0.993      0.994      0.946
  Speed limit (20km/h)         42         42      0.993          1      0.995      0.954
  Speed limit (30km/h)        444        444      0.998      0.998      0.995       0.95
  Speed limit (50km/h)        450        450          1      0.994      0.995      0.937
  Speed limit (60km/h)        282        282          1      0.997      0.995      0.937
  Speed limit (70km/h)        396        396      0.996          1      0.995      0.944
  Speed limit (80km/h)        372        372      0.998      0.997      0.995      0.932
End of speed limit (80km/h)         84         84      0.996          1      0.995      0.925
 Speed limit (100km/h)        288        288      0.999      0.997      0.995      0.948
 Speed limit (120km/h)        282        282          1      0.998      0.995      0.936
            No passing        294        294          1      0.994      0.995       0.95
No passing veh o